In [1]:
# 2d case

### Utils

In [ ]:
import os
import typing
import re

import numpy
import scipy
import scipy.sparse
import scipy.io
from matplotlib import colors


def xy_to_id(x: int, y: int, nelx: int, nely: int, order: str = "F") -> int:
    """
    Map from 2D indices of a node to the flattened 1D index.

    The number of elements is (nelx x nely), and the number of nodes is
    (nelx + 1) x (nely + 1).

    Parameters
    ----------
    x:
        The x-coordinate of the node's positions.
    y:
        The y-coordinate of the node's positions.
    nelx:
        The number of elements in the x-direction.
    nely:
        The number of elements in the y-direction.
    order:
        The order of indecies. "F" for Fortran/column-major order and "C" for
        C/row-major order.

    Returns
    -------
        The index of the node in the flattened version.

    """
    if order == "C":
        return (y * (nelx + 1)) + x
    else:
        return (x * (nely + 1)) + y


def id_to_xy(index: int, nelx: int, nely: int, order: str = "F"
             ) -> typing.Tuple[int, int]:
    """
    Map from a 1D index to 2D indices of a node.

    The number of elements is (nelx x nely), and the number of nodes is
    (nelx + 1) x (nely + 1).

    Parameters
    ----------
    index:
        The 1D index to map to a 2D location.
    nelx:
        The number of elements in the x-direction.
    nely:
        The number of elements in the y-direction.
    order:
        The order of indecies. "F" for Fortran/column-major order and "C" for
        C/row-major order.

    Returns
    -------
        The index of the node in the flattened version.

    """
    if order == "C":
        y = index // (nelx + 1)
        x = index % (nelx + 1)
    else:
        x = index // (nely + 1)
        y = index % (nely + 1)
    return x, y


def deleterowcol(A: scipy.sparse.csc_matrix, delrow: numpy.ndarray,
                 delcol: numpy.ndarray) -> scipy.sparse.csc_matrix:
    """
    Delete the specified rows and columns from csc sparse matrix A.

    Assumes that matrix is in symmetric csc form!

    Parameters
    ----------
    A:
        Matrix
    delrow:
        Row indices to remove.
    delcol:
        Column indices to remove.

    Returns
    -------
        Matrix with rows and columns removed.

    """
    m = A.shape[0]
    keep = numpy.delete(numpy.arange(0, m), delrow)
    A = A[keep, :]
    keep = numpy.delete(numpy.arange(0, m), delcol)
    A = A[:, keep]
    return A


def squared_euclidean(x: numpy.ndarray) -> float:
    """
    Compute the squared euclidean length of x.

    Parameters
    ----------
    x:
        Vector to compute the squared norm of.

    Returns
    -------
        Squared norm of x = :math:`x^Tx`

    """
    return x.T.dot(x)

### BC

In [3]:
from abc import ABC, abstractmethod
class BoundaryConditions():
    """
    Abstract class for boundary conditions to a topology optimization problem.

    Functionalty for geting fixed nodes, forces, and passive elements.

    Attributes
    ----------
    nelx: int
        The number of elements in the x direction.
    nely: int
        The number of elements in the y direction.

    """

    def __init__(self, nelx: int, nely: int):
        """
        Create the boundary conditions with the size of the grid.

        Parameters
        ----------
        nelx:
            The number of elements in the x direction.
        nely:
            The number of elements in the y direction.

        """
        self.nelx = nelx
        self.nely = nely
        self.ndof = 2 * (nelx + 1) * (nely + 1)

    def __str__(self) -> str:
        """Construct a string representation of the boundary conditions."""
        return self.__class__.__name__

    def __format__(self, format_spec) -> str:
        """Construct a formated representation of the boundary conditions."""
        return str(self)

    def __repr__(self) -> str:
        """Construct a representation of the boundary conditions."""
        return "{}(nelx={:d}, nely={:d})".format(
            self.__class__.__name__, self.nelx, self.nely)

    @property
    @abstractmethod
    def fixed_nodes(self):
        """:obj:`numpy.ndarray`: Fixed nodes of the problem."""
        pass

    @property
    @abstractmethod
    def forces(self):
        """:obj:`numpy.ndarray`: Force vector for the problem."""
        pass

    @property
    def passive_elements(self):
        """:obj:`numpy.ndarray`: Passive elements to be set to zero density."""
        return numpy.array([])

    @property
    def active_elements(self):
        """:obj:`numpy.ndarray`: Active elements to be set to full density."""
        return numpy.array([])


class MBBBeamBoundaryConditions(BoundaryConditions):
    """Boundary conditions for the Messerschmitt–Bölkow–Blohm (MBB) beam."""

    @property
    def fixed_nodes(self):
        """:obj:`numpy.ndarray`: Fixed nodes in the bottom corners."""
        dofs = numpy.arange(self.ndof)
        fixed = numpy.union1d(dofs[0:2 * (self.nely + 1):2], numpy.array(
            [2 * (self.nelx + 1) * (self.nely + 1) - 1]))
        return fixed

    @property
    def forces(self):
        """:obj:`numpy.ndarray`: Force vector in the top center."""
        f = numpy.zeros((self.ndof, 1))
        f[1, 0] = -1
        return f

class CantileverBoundaryConditions(BoundaryConditions):
    """Boundary conditions for a cantilever."""

    @property
    def fixed_nodes(self):
        """:obj:`numpy.ndarray`: Fixed nodes on the left."""
        ys = numpy.arange(self.nely + 1)
        lefty_to_id = numpy.vectorize(
            lambda y: xy_to_id(0, y, self.nelx, self.nely))
        ids = lefty_to_id(ys)
        fixed = numpy.union1d(2 * ids, 2 * ids + 1)  # Fix both x and y dof
        return fixed

    @property
    def forces(self):
        """:obj:`numpy.ndarray`: Force vector in the middle right."""
        f = numpy.zeros((self.ndof, 1))
        dof_index = 2 * xy_to_id(
            self.nelx, self.nely // 2, self.nelx, self.nely) + 1
        f[dof_index, 0] = -1
        return f

In [43]:
import torch
from torch import nn, autograd
class Unit(nn.Module):

    def __init__(self, in_N, out_N):
        super(Unit, self).__init__()
        self.in_N = in_N
        self.out_N = out_N
        self.L = nn.Linear(in_N, out_N)

    def forward(self, x):
        x1 = self.L(x)
        x2 = torch.tanh(x1)
        return x2


class NN1(nn.Module):

    def __init__(self, in_N, width, depth, out_N):
        super(NN1, self).__init__()
        self.width = width
        self.in_N = in_N
        self.out_N = out_N
        self.stack = nn.ModuleList()

        self.stack.append(Unit(in_N, width))

        for i in range(depth):
            self.stack.append(Unit(width, width))

        self.stack.append(nn.Linear(width, out_N))

    def forward(self, x):
        # first layer
        for i in range(len(self.stack)):
            x = self.stack[i](x)
        return x


class NN2(nn.Module):
    def __init__(self, in_N, width, depth, out_N):
        super(NN2, self).__init__()
        self.in_N = in_N
        self.width = width
        self.depth = depth
        self.out_N = out_N

        self.stack = nn.ModuleList()

        self.stack.append(nn.Linear(in_N, width))

        for i in range(depth):
            self.stack.append(nn.Linear(width, width))

        self.stack.append(nn.Linear(width, out_N))

    def forward(self, x):
        # first layer
        for i in range(len(self.stack)):
            x = self.stack[i](x)
        return x

def weights_init(m):
    if isinstance(m, (nn.Conv2d, nn.Linear)):
        nn.init.xavier_normal_(m.weight)
        nn.init.constant_(m.bias, 0.0)


### CELL

In [50]:
class MicroStruct:
    def __init__(self) -> None:
        pass
    def getCE_and_CEDrive(self):
        raise NotImplementedError('get_CE must be defined')


class Crossed_2D(MicroStruct):
    def __init__(self, model_L, model_Linear, model_noLinear) -> None:
        self.model_L = torch.load(model_L) 
        self.model_Linear = torch.load(model_Linear)  
        self.model_noLinear = torch.load(model_noLinear) 
        self.alpha = 0.5

    

    def getCE_and_CEDrive(self, x):
        x = np.asanyarray(x).reshape((-1,1))
        x = torch.from_numpy(x.reshape((-1, 1))).float().requires_grad_()
        alpha = self.alpha
        pred_L = self.model_L(x)
        pred_H = alpha * self.model_noLinear(torch.cat((x, pred_L), 1)) +\
              (1 - alpha) * self.model_Linear(torch.cat((x, pred_L), 1))

        dyh_dx = torch.ones_like(pred_H)
        for i in range(3):
            dx_i = autograd.grad(outputs=pred_H[:,i], inputs=x,
                        grad_outputs=torch.ones_like(pred_H[:,i]),
                          retain_graph=True)[0]
            dyh_dx[:,i] = dx_i[:,0]

        
        return pred_H, dyh_dx


    
    


In [52]:
import numpy as np
mircro = Crossed_2D(model_L='model_L.pth',model_Linear='model_Linear.pth',model_noLinear='model_noLinear.pth')
x = np.array([0.5])
mircro.getCE_and_CEDrive(x)

(tensor([[2.2457, 0.8586, 0.8867]], grad_fn=<AddBackward0>),
 tensor([[6.4861, 1.8888, 2.5529]]))

### PROBLEM

In [ ]:
"""Topology optimization problem to solve."""

import abc

import numpy
import scipy.sparse
import scipy.sparse.linalg
import cvxopt
import cvxopt.cholmod




class Problem(abc.ABC):
    """
    Abstract topology optimization problem.

    Attributes
    ----------
    bc: BoundaryConditions
        The boundary conditions for the problem.
    penalty: float
        The SIMP penalty value.
    f: numpy.ndarray
        The right-hand side of the FEM equation (forces).
    u: numpy.ndarray
        The variables of the FEM equation.
    obje: numpy.ndarray
        The per element objective values.

    """

    def __init__(self, bc: BoundaryConditions, micro: MicroStruct):
        """
        Create the topology optimization problem.

        Parameters
        ----------
        bc:
            The boundary conditions of the problem.
        micro:
            The Microstructer used to multiscale topology optimization.

        """
        # Problem size
        self.nelx = bc.nelx
        self.nely = bc.nely
        self.nel = self.nelx * self.nely

        # Count degrees of fredom
        self.ndof = 2 * (self.nelx + 1) * (self.nely + 1)

        # microstructre 
        self.micro = micro

        # BC's and support (half MBB-beam)
        self.bc = bc
        dofs = numpy.arange(self.ndof)
        self.fixed = bc.fixed_nodes
        self.free = numpy.setdiff1d(dofs, self.fixed)

        # RHS and Solution vectors
        self.f = bc.forces
        self.u = numpy.zeros(self.f.shape)

        # Per element objective
        self.obje = numpy.zeros(self.nely * self.nelx)

    def __str__(self) -> str:
        """Create a string representation of the problem."""
        return self.__class__.__name__

    def __format__(self, format_spec) -> str:
        """Create a formated representation of the problem."""
        return str(self)

    def __repr__(self) -> str:
        """Create a representation of the problem."""
        return "{}(bc={!r}, penalty={:g})".format(
            self.__class__.__name__, self.penalty, self.bc)

    def penalize_densities(self, x: numpy.ndarray, drho: numpy.ndarray = None
                           ) -> numpy.ndarray:
        """
        Compute the penalized densties (and optionally its derivative).

        Parameters
        ----------
        x:
            The density variables to penalize.
        drho:
            The derivative of the penealized densities to compute. Only set if
            drho is not None.

        Returns
        -------
        numpy.ndarray
            The penalized densities used for SIMP.

        """
        rho = x**self.penalty
        if drho is not None:
            assert(drho.shape == x.shape)
            drho[:] = rho
            valid = x != 0  # valid values for division
            drho[valid] *= self.penalty / x[valid]
        return rho

    @abc.abstractmethod
    def compute_objective(
            self, xPhys: numpy.ndarray, dobj: numpy.ndarray) -> float:
        """
        Compute objective and its gradient.

        Parameters
        ----------
        xPhys:
            The design variables.
        dobj:
            The gradient of the objective to compute.

        Returns
        -------
        float
            The objective value.

        """
        pass


class ElasticityProblem(Problem):
    """
    Abstract elasticity topology optimization problem.

    Attributes
    ----------
    Emin: float
        The Young's modulus use for the void regions.
    Emax: float
        The Young's modulus use for the solid regions.
    nu: float
        Poisson's ratio of the material.
    f: numpy.ndarray
        The right-hand side of the FEM equation (forces).
    u: numpy.ndarray
        The variables of the FEM equation (displacments).
    nloads: int
        The number of loads applied to the material.

    """

    @staticmethod
    def lk(E: float = 1.0, nu: float = 0.3) -> numpy.ndarray:
        """
        Build the element stiffness matrix.

        Parameters
        ----------
        E:
            The Young's modulus of the material.
        nu:
            The Poisson's ratio of the material.

        Returns
        -------
        numpy.ndarray
            The element stiffness matrix for the material.

        """
        k = numpy.array([
            0.5 - nu / 6., 0.125 + nu / 8., -0.25 - nu / 12.,
            -0.125 + 0.375 * nu, -0.25 + nu / 12., -0.125 - nu / 8., nu / 6.,
            0.125 - 0.375 * nu])
        KE = E / (1 - nu**2) * numpy.array([
            [k[0], k[1], k[2], k[3], k[4], k[5], k[6], k[7]],
            [k[1], k[0], k[7], k[6], k[5], k[4], k[3], k[2]],
            [k[2], k[7], k[0], k[5], k[6], k[3], k[4], k[1]],
            [k[3], k[6], k[5], k[0], k[7], k[2], k[1], k[4]],
            [k[4], k[5], k[6], k[7], k[0], k[1], k[2], k[3]],
            [k[5], k[4], k[3], k[2], k[1], k[0], k[7], k[6]],
            [k[6], k[3], k[4], k[1], k[2], k[7], k[0], k[5]],
            [k[7], k[2], k[1], k[4], k[3], k[6], k[5], k[0]]])
        return KE

    def __init__(self, bc: BoundaryConditions, micro: MicroStruct):
        """
        Create the topology optimization problem.

        Parameters
        ----------
        bc:
            The boundary conditions of the problem.
        penalty:
            The penalty value used to penalize fractional densities in SIMP.

        """
        super().__init__(bc, penalty)
        # Max and min stiffness
        self.Emin = 1e-9
        self.Emax = 1.0

        # FE: Build the index vectors for the for coo matrix format.
        self.nu = 0.3
        self.build_indices()

        # BC's and support (half MBB-beam)
        self.bc = bc
        dofs = numpy.arange(self.ndof)
        self.fixed = bc.fixed_nodes
        self.free = numpy.setdiff1d(dofs, self.fixed)

        # Number of loads
        self.nloads = self.f.shape[1]

    def build_indices(self) -> None:
        """Build the index vectors for the finite element coo matrix format."""
        self.KE = self.lk(E=self.Emax, nu=self.nu)
        self.edofMat = numpy.zeros((self.nelx * self.nely, 8), dtype=int)
        for elx in range(self.nelx):
            for ely in range(self.nely):
                el = ely + elx * self.nely
                n1 = (self.nely + 1) * elx + ely
                n2 = (self.nely + 1) * (elx + 1) + ely
                self.edofMat[el, :] = numpy.array([
                    2 * n1 + 2, 2 * n1 + 3, 2 * n2 + 2, 2 * n2 + 3, 2 * n2,
                    2 * n2 + 1, 2 * n1, 2 * n1 + 1])
        # Construct the index pointers for the coo format
        self.iK = numpy.kron(self.edofMat, numpy.ones((8, 1))).flatten()
        self.jK = numpy.kron(self.edofMat, numpy.ones((1, 8))).flatten()

    def compute_young_moduli(self, x: numpy.ndarray, dE: numpy.ndarray = None
                             ) -> numpy.ndarray:
        """
        Compute the Young's modulus of each element from the densties.

        Optionally compute the derivative of the Young's modulus.

        Parameters
        ----------
        x:
            The density variable of each element.
        dE:
            The derivative of Young's moduli to compute. Only set if dE is not
            None.

        Returns
        -------
        numpy.ndarray
            The elements' Young's modulus.

        """
        drho = None if dE is None else numpy.empty(x.shape)
        rho = self.penalize_densities(x, drho)
        if drho is not None and dE is not None:
            assert(dE.shape == x.shape)
            dE[:] = (self.Emax - self.Emin) * drho
        return (self.Emax - self.Emin) * rho + self.Emin

    def build_K(self, xPhys: numpy.ndarray, remove_constrained: bool = True
                ) -> scipy.sparse.coo_matrix:
        """
        Build the stiffness matrix for the problem.

        Parameters
        ----------
        xPhys:
            The element densisities used to build the stiffness matrix.
        remove_constrained:
            Should the constrained nodes be removed?

        Returns
        -------
        scipy.sparse.coo_matrix
            The stiffness matrix for the mesh.

        """
        sK = ((self.KE.flatten()[numpy.newaxis]).T *
              self.compute_young_moduli(xPhys)).flatten(order='F')
        K = scipy.sparse.coo_matrix(
            (sK, (self.iK, self.jK)), shape=(self.ndof, self.ndof))
        if remove_constrained:
            # Remove constrained dofs from matrix and convert to coo
            K = deleterowcol(K.tocsc(), self.fixed, self.fixed).tocoo()
        return K

    def compute_displacements(self, xPhys: numpy.ndarray) -> numpy.ndarray:
        """
        Compute the displacements given the densities.

        Compute the displacment, :math:`u`, using linear elastic finite
        element analysis (solving :math:`Ku = f` where :math:`K` is the
        stiffness matrix and :math:`f` is the force vector).

        Parameters
        ----------
        xPhys:
            The element densisities used to build the stiffness matrix.

        Returns
        -------
        numpy.ndarray
            The distplacements solve using linear elastic finite element
            analysis.

        """
        # Setup and solve FE problem
        K = self.build_K(xPhys)
        K = cvxopt.spmatrix(
            K.data, K.row.astype(numpy.int), K.col.astype(numpy.int))
        # Solve system
        F = cvxopt.matrix(self.f[self.free, :])
        cvxopt.cholmod.linsolve(K, F)  # F stores solution after solve
        new_u = self.u.copy()
        new_u[self.free, :] = numpy.array(F)[:, :]
        return new_u

    def update_displacements(self, xPhys: numpy.ndarray) -> None:
        """
        Update the displacements of the problem.

        Parameters
        ----------
        xPhys:
            The element densisities used to compute the displacements.

        """
        self.u[:, :] = self.compute_displacements(xPhys)


class ComplianceProblem(ElasticityProblem):
    r"""
    Topology optimization problem to minimize compliance.

    :math:`\begin{aligned}
    \min_{\boldsymbol{\rho}} \quad & \mathbf{f}^T\mathbf{u}\\
    \textrm{subject to}: \quad & \mathbf{K}\mathbf{u} = \mathbf{f}\\
    & \sum_{e=1}^N v_e\rho_e \leq V_\text{frac},
    \quad 0 < \rho_\min \leq \rho_e \leq 1\\
    \end{aligned}`

    where :math:`\mathbf{f}` are the forces, :math:`\mathbf{u}` are the \
    displacements, :math:`\mathbf{K}` is the striffness matrix, and :math:`V`
    is the volume.
    """

    def compute_objective(
            self, xPhys: numpy.ndarray, dobj: numpy.ndarray) -> float:
        r"""
        Compute compliance and its gradient.

        The objective is :math:`\mathbf{f}^{T} \mathbf{u}`. The gradient of
        the objective is

        :math:`\begin{align}
        \mathbf{f}^T\mathbf{u} &= \mathbf{f}^T\mathbf{u} -
        \boldsymbol{\lambda}^T(\mathbf{K}\mathbf{u} - \mathbf{f})\\
        \frac{\partial}{\partial \rho_e}(\mathbf{f}^T\mathbf{u}) &=
        (\mathbf{K}\boldsymbol{\lambda} - \mathbf{f})^T
        \frac{\partial \mathbf u}{\partial \rho_e} +
        \boldsymbol{\lambda}^T\frac{\partial \mathbf K}{\partial \rho_e}
        \mathbf{u}
        = \mathbf{u}^T\frac{\partial \mathbf K}{\partial \rho_e}\mathbf{u}
        \end{align}`

        where :math:`\boldsymbol{\lambda} = \mathbf{u}`.

        Parameters
        ----------
        xPhys:
            The element densities.
        dobj:
            The gradient of compliance.

        Returns
        -------
        float
            The compliance value.

        """
        # Setup and solve FE problem
        self.update_displacements(xPhys)

        obj = 0.0
        dobj[:] = 0.0
        dE = numpy.empty(xPhys.shape)
        E = self.compute_young_moduli(xPhys, dE)
        for i in range(self.nloads):
            ui = self.u[:, i][self.edofMat].reshape(-1, 8)
            self.obje[:] = (ui @ self.KE * ui).sum(1)
            obj += (E * self.obje).sum()
            dobj[:] += -dE * self.obje
        dobj /= float(self.nloads)
        return obj / float(self.nloads)




### TEST

In [ ]:
import numpy as np
x = np.ones((4,3))
x**3

In [ ]:
valid = x != 0
valid